## **Data Preprocessing**

In [ ]:
!pip install transformers datasets torch fastapi uvicorn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 23.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 

In [ ]:
import os
import re
from transformers import GPT2Tokenizer

def preprocess_data(input_file, output_file, tokenizer_name="gpt2"):
    tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_name)
    with open(input_file, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    with open(output_file, 'w', encoding='utf-8') as f:
        for line in lines:
            line = line.lower()

            # Tokenize the line
            token_ids = tokenizer.encode(line, add_special_tokens=False)
            # Convert token IDs back to tokens
            tokenized_line = tokenizer.convert_ids_to_tokens(token_ids)
            # Convert tokens to text and remove special tokens
            processed_line = " ".join(tokenized_line).replace('Ġ', '').replace('Ċ', '').replace('�', '').strip()

            processed_line = re.sub(r'[^a-zA-Z0-9\s,():%&]', '', processed_line)
            processed_line = re.sub(r'\bgu vi\b', 'guvi', processed_line)

            # Remove extra spaces
            processed_line = re.sub(r'\s+', ' ', processed_line)
            # Write the processed line to the output file
            f.write(processed_line + "\n")

input_file = "/content/guvi_data.txt"  # Make sure this path is correct
output_file = "processed_data.txt"
preprocess_data(input_file, output_file)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [ ]:
!pip install accelerate -U

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, TextDataset, DataCollatorForLanguageModeling

# Load pre-trained model and tokenizer
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Create dataset
def load_dataset(file_path, tokenizer, block_size=128):
    return TextDataset(
        tokenizer=tokenizer,
        file_path=file_path,
        block_size=block_size,
    )

train_dataset = load_dataset(output_file, tokenizer)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
)

# Initialize data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

# Fine-tune the model
trainer.train()

# Save the fine-tuned model and tokenizer
model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


Step,Training Loss


('./fine_tuned_model/tokenizer_config.json',
 './fine_tuned_model/special_tokens_map.json',
 './fine_tuned_model/vocab.json',
 './fine_tuned_model/merges.txt',
 './fine_tuned_model/added_tokens.json')

## **Finetuning the GPT-Model**

In [ ]:
#!pip install transformers

from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# Load the fine-tuned model and tokenizer

model_name_or_path = "./fine_tuned_model"  # Use the directory where you saved the model
model = GPT2LMHeadModel.from_pretrained(model_name_or_path)

token_name_or_path = "./fine_tuned_model"  # Use the directory where you saved the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained(token_name_or_path)


# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define the text generation function
def generate_text(model, tokenizer, seed_text, max_length=100, temperature=1.0, num_return_sequences=1):
    # Tokenize the input text
    input_ids = tokenizer.encode(seed_text, return_tensors='pt').to(device)

    # Generate text
    with torch.no_grad():
        output = model.generate(
            input_ids,
            max_length=max_length,
            temperature=temperature,
            num_return_sequences=num_return_sequences,
            do_sample=True,
            top_k=50,
            top_p=0.95,
        )

    # Decode the generated text
    generated_texts = []
    for i in range(num_return_sequences):
        generated_text = tokenizer.decode(output[i], skip_special_tokens=True)
        generated_texts.append(generated_text)

    return generated_texts

In [ ]:
seed_text = "Automation testing course"
generated_texts = generate_text(model, tokenizer, seed_text, max_length=100, temperature=1.0, num_return_sequences=3)
for i, text in enumerate(generated_texts):
    print(f"Generated Text {i + 1}:\n{text}\n")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Text 1:
Automation testing course by the founders of j n an and m il s courses, along with the latest trends in cloud computing and mobile data infrastructure, these courses have helped companies create and deploy highly scalable software solutions by leveraging their extensive expertise in the cloud technologies stack, enabling them to scale projects and reach the right customers fast 

cert ication
guvi, a provider of certificate & validation tools, cert iring experts in a wide range of cloud platforms, and companies in collaboration with the

Generated Text 2:
Automation testing course that combines industry leading technology with high quality industry training you can achieve an industry winning certification from the industry leading certification cert ification company providing this course, ensuring your certification in the industry 
what are the fees associated with this course, including a fee of 5, 000 ( vern acular ) and unlimited downloads
the cost to complete t

In [ ]:
!pip install mysql-connector-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 40.4 MB/s eta 0:00:00


In [ ]:
import mysql.connector

connection = mysql.connector.connect(host="gateway01.ap-southeast-1.prod.aws.tidbcloud.com",port = 4000,
  user = "3bUwYvrRnEGpqae.root",
  password = "ekS17vv8KwPWdZs0",
  database='guvidb')
mycursor = connection.cursor(buffered=True)

In [ ]:
# mycursor.execute('Create database IF NOT EXISTS guvidb')

In [ ]:
# mycursor.execute('create table IF NOT EXISTS guvidb.guvi_table(Username VARCHAR(255)PRIMARY KEY,Password VARCHAR(255))')

In [ ]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.7 MB/s eta 0:00:00


In [ ]:
!pip install pyngrok

## **Streamlit on the GPT Model**

In [ ]:
%%writefile app.py
import streamlit as st
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
import mysql.connector

# Database connection
connection = mysql.connector.connect(
    host="gateway01.ap-southeast-1.prod.aws.tidbcloud.com",
    port=4000,
    user="3bUwYvrRnEGpqae.root",
    password="ekS17vv8KwPWdZs0",
    database="guvidb"
)
mycursor = connection.cursor(buffered=True)

model_name_or_path = "./fine_tuned_model"  # Use the directory where you saved the model
model = GPT2LMHeadModel.from_pretrained(model_name_or_path)

token_name_or_path = "./fine_tuned_model"  # Use the directory where you saved the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained(token_name_or_path)

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define the text generation function
def generate_text(seed_text, max_length=100, temperature=1.0, num_return_sequences=1):
    input_ids = tokenizer.encode(seed_text, return_tensors='pt').to(device)
    with torch.no_grad():
        output = model.generate(
            input_ids,
            max_length=max_length,
            temperature=temperature,
            num_return_sequences=num_return_sequences,
            do_sample=True,
            top_k=50,
            top_p=0.95,
        )
    generated_texts = [tokenizer.decode(output[i], skip_special_tokens=True) for i in range(num_return_sequences)]
    return generated_texts

def insert_credentials(username, password):
    try:
        mycursor.execute('SELECT * FROM guvidb.guvi_table WHERE Username = %s', (username,))
        if mycursor.fetchone():
            st.error("Username already exists. Please choose a different one.")
        else:
            mycursor.execute('INSERT INTO guvidb.guvi_table (Username, Password) VALUES (%s, %s)', (username, password))
            connection.commit()
    except mysql.connector.Error as err:
        st.error(f"Error inserting credentials: {err}")

def authenticate(username, password):
    mycursor.execute('SELECT * FROM guvidb.guvi_table WHERE Username = %s AND Password = %s', (username, password))
    user = mycursor.fetchone()
    return user is not None

def forgot_password(username, new_password):
    try:
        mycursor.execute('SELECT * FROM guvidb.guvi_table WHERE Username = %s', (username,))
        if mycursor.fetchone():
            mycursor.execute('UPDATE guvidb.guvi_table SET Password = %s WHERE Username = %s', (new_password, username))
            connection.commit()
            st.success("Password updated successfully!")
        else:
            st.error("Username not found.")
    except mysql.connector.Error as err:
        st.error(f"Error updating password: {err}")

# Initialize session state variables
if 'page' not in st.session_state:
    st.session_state.page = 'login'
if 'authenticated' not in st.session_state:
    st.session_state.authenticated = False

scrolling_text = """
<h1 style='color:Indigo; font-weight: bold;'>
    <marquee behavior="scroll" direction="left" scrollamount="10">WELCOME TO GUVI GPT-TEXT GENERATOR</marquee>
</h1>
"""
st.markdown(scrolling_text, unsafe_allow_html=True)

# Login Page
if st.session_state.page == 'login':
    st.markdown("<h2 style='color: violet; text-align: center;'>Login</h2>", unsafe_allow_html=True)
    st.markdown("<h3 style='color: red;'>Username</h3>", unsafe_allow_html=True)
    username = st.text_input("Enter Your Username", key="login_username")

    st.markdown("<h3 style='color: red;'>Password</h3>", unsafe_allow_html=True)
    password = st.text_input("Enter Your Password", type="password", key="login_password")

    if st.button("Login"):
        if authenticate(username, password):
            st.session_state.authenticated = True
            st.success("Logged in successfully!")
        else:
            st.error("Incorrect login credentials")

    # If authenticated, show text generation
    if st.session_state.authenticated:
        st.markdown("<h2 style='color: red; text-align: center;'>Text Generation</h2>", unsafe_allow_html=True)
        st.info("Disclaimer: GUVIGPT can make mistakes. The content generated by the model may not always be accurate or appropriate. Please use it responsibly.")
        seed_text = st.text_input("Enter your prompt:")
        max_length = st.slider("Max Length:", min_value=50, max_value=500, value=100)
        temperature = st.slider("Temperature:", min_value=0.1, max_value=2.0, value=1.0)

        if st.button("Generate"):
            if seed_text.strip():
                with st.spinner("Generating text..."):
                    generated_texts = generate_text(seed_text, max_length, temperature)
                    for i, generated_text in enumerate(generated_texts):
                        st.subheader(f"Generated Text {i + 1}")
                        st.write(generated_text)
            else:
                st.warning("Please enter a prompt to generate text.")

    st.markdown("<br>", unsafe_allow_html=True)
    col1, col2 = st.columns(2)

    with col1:
        st.markdown("<h4 style='color: blue;'>Forgot Password?</h4>", unsafe_allow_html=True)
        if st.button("Reset Password"):
            st.session_state.page = 'forgot_password'
            st.experimental_rerun()
    with col2:
        st.markdown("<h4 style='color: blue;'>New user?</h4>", unsafe_allow_html=True)
        if st.button("Go to Sign Up"):
            st.session_state.page = 'signup'
            st.experimental_rerun()

# Forgot Password Page
elif st.session_state.page == 'forgot_password':
    st.markdown("<h2 style='color: yellow;'>Forgot Password</h2>", unsafe_allow_html=True)
    st.markdown("<h3 style='color: blue;'>Forgot Password</h3>", unsafe_allow_html=True)
    with st.form(key='forgot_password_form'):
        forgot_username = st.text_input("Enter your username for password reset")
        new_password = st.text_input("Enter new password", type="password")
        forgot_password_button = st.form_submit_button("Reset Password")

    if forgot_password_button:
        if forgot_username and new_password:
            forgot_password(forgot_username, new_password)
            st.success("Password reset successfully!")
        else:
            st.error("Please provide both username and new password.")

    if st.button("Back to Login"):
        st.session_state.page = 'login'
        st.session_state.authenticated = False
        st.experimental_rerun()

# Sign Up Page
elif st.session_state.page == 'signup':
    st.markdown("<h2 style='color: red;'>Sign Up</h2>", unsafe_allow_html=True)
    st.markdown("<h3 style='color: blue;'>Create a New Account</h3>", unsafe_allow_html=True)
    with st.form(key='create_account_form'):
        new_username = st.text_input("New Username")
        new_password = st.text_input("New Password", type="password")
        create_account_button = st.form_submit_button("Create Account")

    if create_account_button:
        if new_username and new_password:
            insert_credentials(new_username, new_password)
            st.success("Account created successfully!")
        else:
            st.error("Please provide both username and password.")

    if st.button("Back to Login"):
        st.session_state.page = 'login'
        st.session_state.authenticated = False
        st.experimental_rerun()

Writing app.py


In [ ]:
from pyngrok import conf, ngrok
import subprocess
import time

# Authenticate ngrok
conf.get_default().auth_token ="2jOBDcBiwbG1WWJm6sZmRom1n7t_6Jzw2Pm9VuwRb8Fa5bMsy"

# Run the Streamlit app in the background
process = subprocess.Popen(['streamlit', 'run', 'app.py'])

# Give the Streamlit app a few seconds to start

time.sleep(5)

# Expose the Streamlit app to the web using ngrok
public_url = ngrok.connect(addr="8501")
print(f"Public URL: {public_url}")

# Keep the Colab cell running
try:
    while True:
      time.sleep(1)
except KeyboardInterrupt:
    print("Stopping Streamlit app...")
    process.terminate()
    ngrok.disconnect(public_url)
    ngrok.kill()

Public URL: NgrokTunnel: "https://9b41-34-106-26-59.ngrok-free.app" -> "http://localhost:8501"
Stopping Streamlit app...
